## Импорт библиотек и вспомогательные функции

In [1]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz, process
from rapidfuzz import process as rapid_process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import multiprocessing as mp
from functools import partial

def preprocess_text(text):
    """Предобработка текста"""
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def extract_keywords(text):
    """Извлечение ключевых слов из описания"""
    text = preprocess_text(text)
    words = text.split()
    stop_words = {'для', 'из', 'в', 'на', 'с', 'по', 'и', 'или', 'не', 'от', 'до', 'без', 'под', 'над'}
    keywords = [word for word in words if len(word) > 2 and word not in stop_words]
    return set(keywords)

def jaccard_similarity(set1, set2):
    """Вычисление коэффициента Жаккара"""
    if not set1 or not set2:
        return 0
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0

def find_best_match_fuzzy(text, choices, threshold=80):
    """Находит лучшее соответствие с использованием fuzzy matching"""
    matches = process.extract(text, choices, scorer=fuzz.token_set_ratio, limit=5)
    if matches and matches[0][1] >= threshold:
        return matches[0]
    else:
        return (None, 0)



## Функции сопоставления

In [2]:
def match_dataframes_fuzzy(tmc_df, sup_df, threshold=80):
    """Сопоставляет два датафрейма с использованием fuzzy matching"""
    sup_descriptions = sup_df['FULL_NAME/ru_RU'].tolist()
    
    results = []
    total_records = len(tmc_df)
    
    print("Запуск Fuzzy Matching...")
    print(f"Всего записей для обработки: {total_records:,}")
    print("-" * 50)
    
    for idx in range(total_records):
        row = tmc_df.iloc[idx]
        tmc_desc = row['FULL_NAME/ru_RU']
        
        # Простой вывод прогресса каждые 100 записей
        if idx % 100 == 0:
            percent = (idx / total_records) * 100
            print(f"Обработано: {idx:,}/{total_records:,} ({percent:.1f}%)")
        
        match_result = find_best_match_fuzzy(tmc_desc, sup_descriptions, threshold)
        best_match, score = match_result
        
        if best_match:
            sup_match = sup_df[sup_df['FULL_NAME/ru_RU'] == best_match]
            if not sup_match.empty:
                sup_row = sup_match.iloc[0]
                results.append({
                    'TMC_CSCD_ID': row['CSCD_ID'],
                    'TMC_Description': tmc_desc,
                    'SUP_CSCD_ID': sup_row['CSCD_ID'],
                    'SUP_Description': best_match,
                    'Match_Score': score
                })
    
    print(f"Обработано: {total_records:,}/{total_records:,} (100.0%)")
    print("-" * 50)
    return pd.DataFrame(results)

def match_with_tfidf(tmc_df, sup_df, top_k=3, similarity_threshold=0.6):
    """Сопоставление с использованием TF-IDF и косинусного сходства"""
    
    total_records = len(tmc_df)
    print("Предобработка текстов...")
    
    # Предобработка текстов
    tmc_texts = []
    for i in range(total_records):
        desc = tmc_df.iloc[i]['FULL_NAME/ru_RU']
        tmc_texts.append(preprocess_text(desc))
        if i % 100 == 0:
            print(f"TMC: {i:,}/{total_records:,}")
    
    sup_texts = []
    sup_total = len(sup_df)
    for i in range(sup_total):
        desc = sup_df.iloc[i]['FULL_NAME/ru_RU']
        sup_texts.append(preprocess_text(desc))
        if i % 1000 == 0:
            print(f"SUP: {i:,}/{sup_total:,}")
    
    print("Создание TF-IDF матрицы...")
    vectorizer = TfidfVectorizer(min_df=1, max_df=0.8, ngram_range=(1, 2), max_features=10000)
    all_texts = tmc_texts + sup_texts
    tfidf_matrix = vectorizer.fit_transform(all_texts)
    
    tmc_tfidf = tfidf_matrix[:len(tmc_texts)]
    sup_tfidf = tfidf_matrix[len(tmc_texts):]
    
    print("Обучение модели Nearest Neighbors...")
    nbrs = NearestNeighbors(n_neighbors=min(top_k, len(sup_texts)), metric='cosine')
    nbrs.fit(sup_tfidf)
    
    print("Поиск соответствий...")
    distances, indices = nbrs.kneighbors(tmc_tfidf)
    
    results = []
    
    for i in range(len(tmc_texts)):
        dist_list = distances[i]
        idx_list = indices[i]
        
        tmc_desc = tmc_df.iloc[i]['FULL_NAME/ru_RU']
        tmc_id = tmc_df.iloc[i]['CSCD_ID']
        
        if i % 100 == 0:
            percent = (i / total_records) * 100
            print(f"Обработано: {i:,}/{total_records:,} ({percent:.1f}%)")
        
        for j in range(len(dist_list)):
            dist = dist_list[j]
            sup_idx = idx_list[j]
            similarity = 1 - dist
            
            if similarity >= similarity_threshold and sup_idx < len(sup_df):
                sup_row = sup_df.iloc[sup_idx]
                results.append({
                    'TMC_CSCD_ID': tmc_id,
                    'TMC_Description': tmc_desc,
                    'SUP_CSCD_ID': sup_row['CSCD_ID'],
                    'SUP_Description': sup_row['FULL_NAME/ru_RU'],
                    'Similarity_Score': similarity,
                    'Rank': j + 1
                })
    
    print(f"Обработано: {total_records:,}/{total_records:,} (100.0%)")
    return pd.DataFrame(results)

def combined_matching(tmc_df, sup_df, weight_fuzzy=0.6, weight_jaccard=0.4, search_limit=500):
    """Комбинированный подход с использованием нескольких метрик"""
    
    results = []
    total_records = len(tmc_df)
    
    print("Подготовка данных SUP...")
    sup_descriptions = sup_df['FULL_NAME/ru_RU'].tolist()
    
    # Предварительно вычисляем ключевые слова для SUP
    sup_keywords = []
    actual_search_limit = min(search_limit, len(sup_df))
    for i in range(actual_search_limit):
        desc = sup_df.iloc[i]['FULL_NAME/ru_RU']
        sup_keywords.append(extract_keywords(desc))
        if i % 100 == 0:
            print(f"Подготовка SUP: {i:,}/{actual_search_limit:,}")
    
    print("Обработка TMC записей...")
    
    for idx in range(total_records):
        tmc_row = tmc_df.iloc[idx]
        tmc_desc = tmc_row['FULL_NAME/ru_RU']
        tmc_keywords = extract_keywords(tmc_desc)
        
        best_match = None
        best_score = 0
        best_sup_idx = -1
        
        for sup_idx in range(actual_search_limit):
            sup_row = sup_df.iloc[sup_idx]
            sup_desc = sup_row['FULL_NAME/ru_RU']
            
            # Fuzzy matching score
            fuzzy_score = fuzz.token_set_ratio(tmc_desc, sup_desc) / 100
            
            # Jaccard similarity
            jaccard_score = jaccard_similarity(tmc_keywords, sup_keywords[sup_idx])
            
            # Комбинированный score
            combined_score = (weight_fuzzy * fuzzy_score + 
                            weight_jaccard * jaccard_score)
            
            if combined_score > best_score:
                best_score = combined_score
                best_match = sup_row
                best_sup_idx = sup_idx
        
        if idx % 100 == 0:
            percent = (idx / total_records) * 100
            matches_found = len(results)
            print(f"Обработано: {idx:,}/{total_records:,} ({percent:.1f}%), найдено: {matches_found}")
        
        if best_match is not None and best_score > 0.5:
            results.append({
                'TMC_CSCD_ID': tmc_row['CSCD_ID'],
                'TMC_Description': tmc_desc,
                'SUP_CSCD_ID': best_match['CSCD_ID'],
                'SUP_Description': best_match['FULL_NAME/ru_RU'],
                'Combined_Score': best_score,
                'Fuzzy_Score': fuzz.token_set_ratio(tmc_desc, best_match['FULL_NAME/ru_RU']) / 100,
                'Jaccard_Score': jaccard_similarity(tmc_keywords, sup_keywords[best_sup_idx])
            })
    
    print(f"Обработано: {total_records:,}/{total_records:,} (100.0%)")
    return pd.DataFrame(results)

def parallel_fuzzy_match(args):
    """Параллельная обработка чанков данных"""
    tmc_chunk, sup_descriptions, threshold, chunk_id, total_chunks = args
    results = []
    
    for tmc_desc in tmc_chunk:
        match = rapid_process.extractOne(
            tmc_desc, 
            sup_descriptions, 
            scorer=fuzz.token_set_ratio, 
            score_cutoff=threshold
        )
        if match:
            results.append((tmc_desc, match[0], match[1]))
    
    return results

def parallel_matching(tmc_df, sup_df, threshold=75, n_workers=4):
    """Параллельное сопоставление для ускорения обработки"""
    
    tmc_descriptions = tmc_df['FULL_NAME/ru_RU'].tolist()
    sup_descriptions = sup_df['FULL_NAME/ru_RU'].tolist()
    total_records = len(tmc_descriptions)
    
    chunk_size = max(1, len(tmc_descriptions) // n_workers)
    chunks = [tmc_descriptions[i:i + chunk_size] for i in range(0, len(tmc_descriptions), chunk_size)]
    
    chunk_args = []
    for i, chunk in enumerate(chunks):
        chunk_args.append((chunk, sup_descriptions, threshold, i, len(chunks)))
    
    print(f"Параллельная обработка с {n_workers} workers...")
    print(f"Всего записей: {total_records:,}")
    print(f"Количество чанков: {len(chunks)}")
    
    all_results = []
    with mp.Pool(n_workers) as pool:
        for i, result in enumerate(pool.imap(parallel_fuzzy_match, chunk_args)):
            all_results.append(result)
            print(f"Обработан чанк {i+1}/{len(chunks)}")
    
    flat_results = [item for sublist in all_results for item in sublist]
    
    print("Формирование итогового DataFrame...")
    results_df = []
    
    for tmc_desc, sup_desc, score in flat_results:
        tmc_match = tmc_df[tmc_df['FULL_NAME/ru_RU'] == tmc_desc]
        sup_match = sup_df[sup_df['FULL_NAME/ru_RU'] == sup_desc]
        
        if not tmc_match.empty and not sup_match.empty:
            tmc_row = tmc_match.iloc[0]
            sup_row = sup_match.iloc[0]
            
            results_df.append({
                'TMC_CSCD_ID': tmc_row['CSCD_ID'],
                'TMC_Description': tmc_desc,
                'SUP_CSCD_ID': sup_row['CSCD_ID'],
                'SUP_Description': sup_desc,
                'Match_Score': score
            })
    
    return pd.DataFrame(results_df)


## Функция запуска методов и Excel функции

In [3]:
def match_with_progress(tmc_df, sup_df, method='fuzzy', **kwargs):
    """
    Универсальная функция запуска методов
    """
    from time import time
    start_time = time()
    
    total_tmc = len(tmc_df)
    total_sup = len(sup_df)
    
    print(f"Начало обработки: {total_tmc:,} TMC записей vs {total_sup:,} SUP записей")
    print(f"Метод: {method}")
    print("-" * 60)
    
    if method == 'fuzzy':
        results = match_dataframes_fuzzy(tmc_df, sup_df, **kwargs)
    elif method == 'tfidf':
        results = match_with_tfidf(tmc_df, sup_df, **kwargs)
    elif method == 'combined':
        results = combined_matching(tmc_df, sup_df, **kwargs)
    elif method == 'parallel':
        results = parallel_matching(tmc_df, sup_df, **kwargs)
    else:
        raise ValueError("Неизвестный метод. Доступные: 'fuzzy', 'tfidf', 'combined', 'parallel'")
    
    end_time = time()
    processing_time = end_time - start_time
    
    print("-" * 60)
    print("ОБРАБОТКА ЗАВЕРШЕНА")
    print(f"Время обработки: {processing_time:.2f} секунд")
    print(f"Найдено соответствий: {len(results):,}")
    print(f"Процент сопоставления: {len(results)/total_tmc*100:.1f}%")
    
    if len(results) > 0:
        score_column = results.columns[-1]
        avg_score = results[score_column].mean()
        max_score = results[score_column].max()
        min_score = results[score_column].min()
        print(f"Средняя оценка соответствия: {avg_score:.3f}")
        print(f"Лучшее соответствие: {max_score:.3f}")
        print(f"Худшее соответствие: {min_score:.3f}")
    
    return results

def save_results_to_excel(results, tmc_df, sup_df, filename, method_name):
    """
    Сохраняет результаты одного метода в Excel файл
    """
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        # Основные результаты
        results.to_excel(writer, sheet_name='Сопоставления', index=False)
        
        if len(results) > 0:
            score_column = results.columns[-1]
            
            # Статистика
            stats_data = {
                'Метрика': [
                    'Всего записей TMC',
                    'Всего записей SUP', 
                    'Найдено соответствий',
                    'Процент сопоставления',
                    'Средняя оценка',
                    'Максимальная оценка',
                    'Минимальная оценка'
                ],
                'Значение': [
                    len(tmc_df),
                    len(sup_df),
                    len(results),
                    f"{len(results)/len(tmc_df)*100:.2f}%",
                    f"{results[score_column].mean():.3f}",
                    f"{results[score_column].max():.3f}",
                    f"{results[score_column].min():.3f}"
                ]
            }
            stats_df = pd.DataFrame(stats_data)
            stats_df.to_excel(writer, sheet_name='Статистика', index=False)
            
            # Топ-10 лучших соответствий
            top_10 = results.nlargest(10, score_column)
            top_10.to_excel(writer, sheet_name='Топ-10 соответствий', index=False)

def create_summary_excel(all_results, tmc_df, sup_df):
    """
    Создает сводный Excel файл со всеми результатами методов
    """
    with pd.ExcelWriter('all_methods_comparison.xlsx', engine='openpyxl') as writer:
        
        # Сравнение методов
        comparison_data = []
        for method_name, results in all_results.items():
            if len(results) > 0:
                score_column = results.columns[-1]
                comparison_data.append({
                    'Метод': method_name.upper(),
                    'Всего_совпадений': len(results),
                    'Процент_совпадений': f"{(len(results)/len(tmc_df)*100):.2f}%",
                    'Средняя_оценка': f"{results[score_column].mean():.3f}",
                    'Максимальная_оценка': f"{results[score_column].max():.3f}",
                    'Минимальная_оценка': f"{results[score_column].min():.3f}"
                })
        
        comparison_df = pd.DataFrame(comparison_data)
        comparison_df.to_excel(writer, sheet_name='Сравнение методов', index=False)
        
        # Результаты каждого метода
        for method_name, results in all_results.items():
            sheet_name = method_name[:31]
            results.to_excel(writer, sheet_name=sheet_name, index=False)
        
        # Общая статистика
        stats_summary = []
        for method_name, results in all_results.items():
            if len(results) > 0:
                score_column = results.columns[-1]
                stats_summary.append({
                    'Метод': method_name.upper(),
                    'Всего записей TMC': len(tmc_df),
                    'Всего записей SUP': len(sup_df),
                    'Найдено соответствий': len(results),
                    'Процент успеха': f"{(len(results)/len(tmc_df)*100):.2f}%",
                    'Средняя оценка': results[score_column].mean()
                })
        
        stats_summary_df = pd.DataFrame(stats_summary)
        stats_summary_df.to_excel(writer, sheet_name='Общая статистика', index=False)


## Загрузка данных

In [4]:
# Загрузка данных
print("📥 Загрузка данных...")
tmc_df = pd.read_excel('ОКПД_1.xlsx')
sup_df = pd.read_excel('ОКПД_2.xlsx')

📥 Загрузка данных...


## Основной скрипт выполнения

In [5]:
# Основной скрипт
print("🚀 ЗАПУСК СИСТЕМЫ СРАВНЕНИЯ НОМЕНКЛАТУР")
print("=" * 70)
print(f"✅ TMC записей: {len(tmc_df):,}")
print(f"✅ SUP записей: {len(sup_df):,}")
print()

# Определяем методы для тестирования
methods = [
    ('fuzzy', {'threshold': 80}),
    ('tfidf', {'top_k': 3, 'similarity_threshold': 0.5}),
    ('combined', {'search_limit': 200, 'weight_fuzzy': 0.7, 'weight_jaccard': 0.3})
]

all_results = {}

for method_name, params in methods:
    print("\n" + "=" * 70)
    print(f"🔍 ЗАПУСК МЕТОДА: {method_name.upper()}")
    print("=" * 70)
    
    try:
        results = match_with_progress(tmc_df, sup_df, method=method_name, **params)
        all_results[method_name] = results
        
        # Сохранение результатов в Excel
        filename = f'{method_name}_matching_results.xlsx'
        save_results_to_excel(results, tmc_df, sup_df, filename, method_name)
        print(f"💾 Результаты сохранены в: {filename}")
        
    except Exception as e:
        print(f"❌ Ошибка в методе {method_name}: {e}")
        import traceback
        traceback.print_exc()
        continue

print("\n🎉 ВСЕ МЕТОДЫ ЗАВЕРШЕНЫ!")

# Вывод статистики
print("\n📊 СВОДНАЯ СТАТИСТИКА:")
print("=" * 50)

for method_name, results in all_results.items():
    if len(results) > 0:
        score_column = results.columns[-1]
        avg_score = results[score_column].mean()
        print(f"{method_name.upper():<12} | Совпадений: {len(results):>6,} | Средняя оценка: {avg_score:.3f}")

# Создание сводного отчета
if all_results:
    print(f"\n📊 СОЗДАНИЕ СВОДНОГО ОТЧЕТА...")
    create_summary_excel(all_results, tmc_df, sup_df)
    print("💾 Сводный отчет сохранен в: all_methods_comparison.xlsx")

print(f"\n💡 Всего обработано методов: {len(all_results)} из {len(methods)}")


🚀 ЗАПУСК СИСТЕМЫ СРАВНЕНИЯ НОМЕНКЛАТУР
✅ TMC записей: 351
✅ SUP записей: 15,064


🔍 ЗАПУСК МЕТОДА: FUZZY
Начало обработки: 351 TMC записей vs 15,064 SUP записей
Метод: fuzzy
------------------------------------------------------------
Запуск Fuzzy Matching...
Всего записей для обработки: 351
--------------------------------------------------
Обработано: 0/351 (0.0%)
Обработано: 100/351 (28.5%)
Обработано: 200/351 (57.0%)
Обработано: 300/351 (85.5%)
Обработано: 351/351 (100.0%)
--------------------------------------------------
------------------------------------------------------------
ОБРАБОТКА ЗАВЕРШЕНА
Время обработки: 67.07 секунд
Найдено соответствий: 262
Процент сопоставления: 74.6%
Средняя оценка соответствия: 91.275
Лучшее соответствие: 100.000
Худшее соответствие: 81.000
💾 Результаты сохранены в: fuzzy_matching_results.xlsx

🔍 ЗАПУСК МЕТОДА: TFIDF
Начало обработки: 351 TMC записей vs 15,064 SUP записей
Метод: tfidf
------------------------------------------------------------


## Функция сравнения результатов

In [6]:
def compare_results(all_results, tmc_df):
    """Сравнивает результаты разных методов"""
    comparison_data = []
    
    for method_name, results in all_results.items():
        if len(results) > 0:
            score_column = results.columns[-1]
            avg_score = results[score_column].mean()
            match_rate = len(results) / len(tmc_df) * 100
            
            comparison_data.append({
                'Method': method_name.upper(),
                'Total_Matches': len(results),
                'Match_Rate_Percent': match_rate,
                'Average_Score': avg_score,
                'Best_Match': results[score_column].max(),
                'Worst_Match': results[score_column].min()
            })
    
    comparison_df = pd.DataFrame(comparison_data)
    print("\n📋 СРАВНЕНИЕ МЕТОДОВ:")
    print("=" * 80)
    print(comparison_df.to_string(index=False, float_format='%.3f'))
    
    # Сохраняем сравнение
    comparison_df.to_excel('methods_comparison.xlsx', index=False, engine='openpyxl')
    print(f"\n💾 Сравнение методов сохранено в: methods_comparison.xlsx")
    
    return comparison_df

# Запускаем сравнение если есть результаты
if all_results:
    comparison = compare_results(all_results, tmc_df)

print("\n🎯 АНАЛИЗ ЗАВЕРШЕН!")



📋 СРАВНЕНИЕ МЕТОДОВ:
  Method  Total_Matches  Match_Rate_Percent  Average_Score  Best_Match  Worst_Match
   FUZZY            262              74.644         91.275     100.000       81.000
   TFIDF            565             160.969          1.704       3.000        1.000
COMBINED              4               1.140          0.424       0.500        0.250

💾 Сравнение методов сохранено в: methods_comparison.xlsx

🎯 АНАЛИЗ ЗАВЕРШЕН!


## Поиск конкретной номенклатуры

In [21]:
# Загрузка данных
print("📥 Загрузка данных...")
tmc_df = pd.read_excel('TMC.xlsx')
sup_df = pd.read_excel('SUP.xlsx')
sup_df = pd.concat([tmc_df,sup_df],ignore_index=True)
print("📥 Данные загружены")

📥 Загрузка данных...


In [22]:
def search_nomenclature_by_description(description, tmc_df, sup_df, threshold=70, top_k=5):
    """
    Ищет номенклатуру по описанию тремя методами
    """
    print(f"🔍 ПОИСК НОМЕНКЛАТУРЫ: '{description}'")
    print("=" * 80)
    
    results = {
        'fuzzy': [],
        'tfidf': [],
        'combined': []
    }
    
    # Метод 1: Fuzzy Matching
    print("\n1. FUZZY MATCHING:")
    print("-" * 40)
    sup_descriptions = sup_df['FULL_NAME/ru_RU'].tolist()
    fuzzy_matches = process.extract(description, sup_descriptions, scorer=fuzz.token_set_ratio, limit=top_k)
    
    for match_desc, score in fuzzy_matches:
        if score >= threshold:
            sup_match = sup_df[sup_df['FULL_NAME/ru_RU'] == match_desc]
            if not sup_match.empty:
                sup_row = sup_match.iloc[0]
                results['fuzzy'].append({
                    'SUP_CSCD_ID': sup_row['CSCD_ID'],
                    'SUP_Description': match_desc,
                    'Score': score,
                    'Method': 'Fuzzy'
                })
                print(f"   📍 Оценка: {score:>3}% | ID: {sup_row['CSCD_ID']}")
                print(f"      Описание: {match_desc}")
    
    # Метод 2: TF-IDF
    print("\n2. TF-IDF MATCHING:")
    print("-" * 40)
    
    # Подготовка текстов
    search_text = preprocess_text(description)
    sup_texts = [preprocess_text(desc) for desc in sup_df['FULL_NAME/ru_RU']]
    
    # Создание TF-IDF матрицы
    vectorizer = TfidfVectorizer(min_df=1, max_df=0.8, ngram_range=(1, 2))
    all_texts = [search_text] + sup_texts
    tfidf_matrix = vectorizer.fit_transform(all_texts)
    
    # Разделение матрицы
    search_tfidf = tfidf_matrix[0]
    sup_tfidf = tfidf_matrix[1:]
    
    # Вычисление косинусного сходства
    from sklearn.metrics.pairwise import cosine_similarity
    similarities = cosine_similarity(search_tfidf, sup_tfidf).flatten()
    
    # Получение топ-K результатов
    top_indices = similarities.argsort()[-top_k:][::-1]
    
    for idx in top_indices:
        similarity = similarities[idx]
        if similarity >= 0.3:  # Порог для TF-IDF
            sup_row = sup_df.iloc[idx]
            results['tfidf'].append({
                'SUP_CSCD_ID': sup_row['CSCD_ID'],
                'SUP_Description': sup_row['FULL_NAME/ru_RU'],
                'Score': similarity * 100,
                'Method': 'TF-IDF'
            })
            print(f"   📍 Оценка: {similarity*100:>5.1f}% | ID: {sup_row['CSCD_ID']}")
            print(f"      Описание: {sup_row['FULL_NAME/ru_RU']}")
    
    # Метод 3: Комбинированный
    print("\n3. COMBINED MATCHING:")
    print("-" * 40)
    
    search_keywords = extract_keywords(description)
    combined_results = []
    
    for idx, sup_row in sup_df.iterrows():
        sup_desc = sup_row['FULL_NAME/ru_RU']
        sup_keywords = extract_keywords(sup_desc)
        
        # Fuzzy score
        fuzzy_score = fuzz.token_set_ratio(description, sup_desc) / 100
        
        # Jaccard similarity
        jaccard_score = jaccard_similarity(search_keywords, sup_keywords)
        
        # Combined score (взвешенное среднее)
        combined_score = (0.6 * fuzzy_score + 0.4 * jaccard_score) * 100
        
        if combined_score >= threshold:
            combined_results.append({
                'sup_row': sup_row,
                'combined_score': combined_score,
                'fuzzy_score': fuzzy_score * 100,
                'jaccard_score': jaccard_score * 100
            })
    
    # Сортируем и берем топ-K
    combined_results.sort(key=lambda x: x['combined_score'], reverse=True)
    
    for i, result in enumerate(combined_results[:top_k]):
        sup_row = result['sup_row']
        results['combined'].append({
            'SUP_CSCD_ID': sup_row['CSCD_ID'],
            'SUP_Description': sup_row['FULL_NAME/ru_RU'],
            'Score': result['combined_score'],
            'Method': 'Combined',
            'Fuzzy_Score': result['fuzzy_score'],
            'Jaccard_Score': result['jaccard_score']
        })
        print(f"   📍 Оценка: {result['combined_score']:>5.1f}% | ID: {sup_row['CSCD_ID']}")
        print(f"      Fuzzy: {result['fuzzy_score']:.1f}% | Jaccard: {result['jaccard_score']:.1f}%")
        print(f"      Описание: {sup_row['FULL_NAME/ru_RU']}")
    
    return results

def save_search_results_to_excel(search_description, results, filename):
    """
    Сохраняет результаты поиска в Excel файл
    """
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        
        # Сводная информация о поиске
        summary_data = {
            'Параметр': ['Поисковый запрос', 'Дата поиска', 'Всего найдено Fuzzy', 'Всего найдено TF-IDF', 'Всего найдено Combined'],
            'Значение': [
                search_description,
                pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
                len(results['fuzzy']),
                len(results['tfidf']),
                len(results['combined'])
            ]
        }
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Сводка', index=False)
        
        # Результаты Fuzzy Matching
        if results['fuzzy']:
            fuzzy_df = pd.DataFrame(results['fuzzy'])
            fuzzy_df.to_excel(writer, sheet_name='Fuzzy_Results', index=False)
        
        # Результаты TF-IDF
        if results['tfidf']:
            tfidf_df = pd.DataFrame(results['tfidf'])
            tfidf_df.to_excel(writer, sheet_name='TFIDF_Results', index=False)
        
        # Результаты Combined
        if results['combined']:
            combined_df = pd.DataFrame(results['combined'])
            combined_df.to_excel(writer, sheet_name='Combined_Results', index=False)
        
        # Все результаты вместе
        all_results = []
        for method in ['fuzzy', 'tfidf', 'combined']:
            for result in results[method]:
                all_results.append(result)
        
        if all_results:
            all_df = pd.DataFrame(all_results)
            all_df.to_excel(writer, sheet_name='Все_результаты', index=False)
            
            # Топ-10 лучших результатов по всем методам
            top_10 = all_df.nlargest(10, 'Score')
            top_10.to_excel(writer, sheet_name='Топ-10', index=False)

# Функция для интерактивного поиска
def interactive_search():
    """
    Интерактивная функция для поиска номенклатур
    """
    print("🎯 ИНТЕРАКТИВНЫЙ ПОИСК НОМЕНКЛАТУРЫ")
    print("=" * 50)
    
    while True:
        print("\nВведите описание номенклатуры для поиска (или 'выход' для завершения):")
        search_query = input("> ").strip()
        
        if search_query.lower() in ['выход', 'exit', 'quit']:
            print("Завершение работы...")
            break
        
        if not search_query:
            print("❌ Пустой запрос. Попробуйте снова.")
            continue
        
        print(f"\n🔍 Поиск: '{search_query}'")
        print("⏳ Обработка...")
        
        try:
            # Выполняем поиск
            results = search_nomenclature_by_description(search_query, tmc_df, sup_df, threshold=70, top_k=5)
            
            # Сохраняем результаты
            timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
            filename = f'search_results_{timestamp}.xlsx'
            save_search_results_to_excel(search_query, results, filename)
            
            print(f"\n💾 Результаты поиска сохранены в: {filename}")
            
            # Сводная статистика
            total_matches = len(results['fuzzy']) + len(results['tfidf']) + len(results['combined'])
            print(f"\n📊 Сводная статистика:")
            print(f"   • Fuzzy Matching: {len(results['fuzzy'])} совпадений")
            print(f"   • TF-IDF Matching: {len(results['tfidf'])} совпадений")
            print(f"   • Combined Matching: {len(results['combined'])} совпадений")
            print(f"   • Всего найдено: {total_matches} совпадений")
            
        except Exception as e:
            print(f"❌ Ошибка при поиске: {e}")
            import traceback
            traceback.print_exc()

# Запуск интерактивного поиска
print("🎯 ГОТОВ К ПОИСКУ КОНКРЕТНЫХ НОМЕНКЛАТУР!")
print("=" * 60)




🎯 ГОТОВ К ПОИСКУ КОНКРЕТНЫХ НОМЕНКЛАТУР!


In [24]:
test_descriptions = [
    "Теплообменник кожухотрубчатый 500 ТПГ-4,0-М1/25Г-3-К-2-ХЛ-И",
    "Теплообменник кожухотрубчатый 1-EW-3001/1,2"
]

In [28]:
# Пример поиска конкретной номенклатуры

print("\n📋 Примеры для быстрого тестирования:")
for i, desc in enumerate(test_descriptions, 1):
    print(f"   {i}. {desc}")

print("\nВыберите вариант:")
print("1. Запустить интерактивный поиск")
print("2. Протестировать на примере 'Труба обсадная'")
print("3. Ввести свой запрос")

choice = input("Ваш выбор (1-3): ").strip()

if choice == "1":
    interactive_search()
elif choice == "2":
    search_query = "Труба обсадная"
    print(f"\n🔍 ТЕСТОВЫЙ ПОИСК: '{search_query}'")
    results = search_nomenclature_by_description(search_query, tmc_df, sup_df, threshold=70, top_k=5)
    
    # Сохраняем результаты
    filename = f'search_example_{pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")}.xlsx'
    save_search_results_to_excel(search_query, results, filename)
    print(f"\n💾 Результаты сохранены в: {filename}")
    
elif choice == "3":
    custom_query = input("Введите описание для поиска: ").strip()
    if custom_query:
        results = search_nomenclature_by_description(custom_query, tmc_df, sup_df, threshold=70, top_k=5)
        filename = f'search_custom_{pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")}.xlsx'
        save_search_results_to_excel(custom_query, results, filename)
        print(f"\n💾 Результаты сохранены в: {filename}")
    else:
        print("❌ Пустой запрос.")
else:
    print("❌ Неверный выбор.")


📋 Примеры для быстрого тестирования:
   1. Теплообменник кожухотрубчатый 500 ТПГ-4,0-М1/25Г-3-К-2-ХЛ-И
   2. Теплообменник кожухотрубчатый 1-EW-3001/1,2

Выберите вариант:
1. Запустить интерактивный поиск
2. Протестировать на примере 'Труба обсадная'
3. Ввести свой запрос


🔍 ПОИСК НОМЕНКЛАТУРЫ: 'Костюм мужской размер 50 зимний утепленный'

1. FUZZY MATCHING:
----------------------------------------
   📍 Оценка: 100% | ID: 2140291
      Описание: Костюм зимний
   📍 Оценка: 100% | ID: 3316202
      Описание: КОСТЮМ УТЕПЛЕННЫЙ
   📍 Оценка:  91% | ID: 3267123
      Описание: Костюм зимний мужской б/у
   📍 Оценка:  91% | ID: 1527594
      Описание: Костюм зимний мужской МЧС
   📍 Оценка:  91% | ID: 3427670
      Описание: Костюм мужской зимний МЧС

2. TF-IDF MATCHING:
----------------------------------------
   📍 Оценка:  33.4% | ID: 3073156
      Описание: Костюм х/б мужской

3. COMBINED MATCHING:
----------------------------------------
   📍 Оценка:  78.6% | ID: 3267123
      Fuzzy: 91.0% | Jaccard: 60.0%
      Описание: Костюм зимний мужской б/у
   📍 Оценка:  77.4% | ID: 3052822
      Fuzzy: 89.0% | Jaccard: 60.0%
      Описание: Костюм мужской зимний Е-40
   📍 Оценка:  76.0% | ID: 2140291
      Fuzzy: 100.0% | Jaccard: 40.0%
      Описание: Костюм зимний
 

## Дополнительные утилиты для поиска

In [20]:
def quick_search(description, top_k=3):
    """
    Быстрый поиск по одному описанию с выводом в консоль
    """
    print(f"\n🔍 БЫСТРЫЙ ПОИСК: '{description}'")
    print("=" * 60)
    
    sup_descriptions = sup_df['FULL_NAME/ru_RU'].tolist()
    
    # Простой Fuzzy search
    matches = process.extract(description, sup_descriptions, scorer=fuzz.token_set_ratio, limit=top_k)
    
    print("🎯 ЛУЧШИЕ СООТВЕТСТВИЯ:")
    for i, (match_desc, score) in enumerate(matches, 1):
        sup_match = sup_df[sup_df['FULL_NAME/ru_RU'] == match_desc]
        if not sup_match.empty:
            sup_row = sup_match.iloc[0]
            print(f"\n{i}. 📍 Оценка: {score}%")
            print(f"   ID: {sup_row['CSCD_ID']}")
            print(f"   Описание: {match_desc}")
            
            # Проверяем, есть ли в TMC
            tmc_match = tmc_df[tmc_df['FULL_NAME/ru_RU'].str.contains(description, case=False, na=False)]
            if not tmc_match.empty:
                print(f"   ✅ Есть в TMC: {len(tmc_match)} записей")
            else:
                print(f"   ❌ Нет в TMC")

# Примеры быстрого поиска
print("🚀 БЫСТРЫЙ ПОИСК - ТЕСТОВЫЕ ПРИМЕРЫ")
print("=" * 50)

test_queries = [
    "Теплообменник кожухотрубчатый Т-119",
    "Труба обсадная 114"
]

for query in test_queries:
    quick_search(query)
    print("\n" + "-" * 50)

print("\n🎯 Для дополнительного поиска используйте функцию interactive_search()")


🚀 БЫСТРЫЙ ПОИСК - ТЕСТОВЫЕ ПРИМЕРЫ

🔍 БЫСТРЫЙ ПОИСК: 'Теплообменник кожухотрубчатый Т-119'
🎯 ЛУЧШИЕ СООТВЕТСТВИЯ:

1. 📍 Оценка: 100%
   ID: 2211674
   Описание: Теплообменник
   ✅ Есть в TMC: 1 записей

2. 📍 Оценка: 100%
   ID: 2218819
   Описание: Теплообменник кожухотрубчатый
   ✅ Есть в TMC: 1 записей

3. 📍 Оценка: 97%
   ID: 1471209
   Описание: Теплообменник кожухотрубчатый Т-131
   ✅ Есть в TMC: 1 записей

--------------------------------------------------

🔍 БЫСТРЫЙ ПОИСК: 'Труба обсадная 114'
🎯 ЛУЧШИЕ СООТВЕТСТВИЯ:

1. 📍 Оценка: 100%
   ID: 3189715
   Описание: Труба обсадная ОТТМ 114,3х6,35-Дс ТУ 14-3Р-30-2015 с муфтами
   ❌ Нет в TMC

2. 📍 Оценка: 100%
   ID: 3012205
   Описание: Труба обсадная BC 114,3х7,37-Ес ТУ 14-3Р-30-2015 с муфтами
   ❌ Нет в TMC

3. 📍 Оценка: 100%
   ID: 2313907
   Описание: Труба обсадная 114,3х7,37-R3-N80 тип Q-S-PSL-1 ГОСТ 31446-2017
   ❌ Нет в TMC

--------------------------------------------------

🎯 Для дополнительного поиска используйте функцию 